In [1]:
# Mnist Classification using CNN

In [2]:
import torch
import torch.nn as nn

In [3]:
inputs = torch.Tensor(1, 1, 28, 28) # 배치크기 , 채널, 높이, 너비
print(' 텐서의 크기 : {} '.format(inputs.shape))

 텐서의 크기 : torch.Size([1, 1, 28, 28]) 


In [5]:
conv1 = nn.Conv2d(1, 32, 3, padding = 1) # input 1채널, output 32채널, kernel_size = 3, padding = 1
print(conv1)

conv2 = nn.Conv2d(32, 64, kernel_size = 3, padding = 1)
print(conv2)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [6]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [7]:
out = conv1(inputs)
print(out.shape) # 28 x 28 x 32(채널)

out = pool(out)
print(out.shape)

out = conv2(out)
print(out.shape)

out = pool(out)
print(out.shape) # 첫번째 1 (out.size(0) == 1) 은 배치 사이즈 차원

torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])
torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])


In [8]:
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [9]:
fc = nn.Linear(3136, 10) #input_dim = 3136, output_dim = 10
out = fc(out)
print(out.shape)

torch.Size([1, 10])


In [11]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [13]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [15]:
mnist_train = dsets.MNIST(root = './MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)

mnist_test = dsets.MNIST(root = './MNIST_data/',
                        train = False,
                        transform = transforms.ToTensor(),
                        download = True) 

In [16]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size = batch_size,
                                         shuffle = True,
                                         drop_last = True)

In [18]:
class  CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
         # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out
        

In [19]:
model = CNN().to(device)

In [20]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [21]:
total_batch = len(data_loader)

for epoch in range(total_batch + 1):
    avg_cost = 0
    
    for X,Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.220153883
[Epoch:    2] cost = 0.0609047301
[Epoch:    3] cost = 0.0460579768
[Epoch:    4] cost = 0.0366953798


KeyboardInterrupt: 

In [22]:
print(avg_cost)

tensor(0.0148, grad_fn=<AddBackward0>)


In [24]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuraty = correct_prediction.float().mean()
    print(accuraty.item())

0.9843000173568726
